In [1]:
# dataset:https://www.kaggle.com/grouplens/movielens-20m-dataset

In [2]:
import pandas as pd
import numpy as np
import socket
from socket import socket

In [3]:
import findspark
findspark.init()
import pyspark

In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, expr
from pyspark.sql import udf
from pyspark.sql.types import BooleanType
from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id,row_number

import pyspark.sql.functions as f

from pyspark.sql.functions import collect_set
from pyspark.sql.functions import countDistinct
from pyspark.sql import Window
from pyspark.sql.functions import col, mean, sum, desc
from pyspark.sql.functions import when
from pyspark.sql.functions import explode, split #exclude null
from pyspark.sql.functions import lit
from pyspark.sql.functions import explode_outer #include null
from pyspark.sql.functions import posexplode_outer
from pyspark.sql import Window

In [5]:
# sqlContext = SQLContext(sc)
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName("csvload").getOrCreate()

In [6]:
genome_scores_fp = r"/Users/carolliu/Desktop/project_new/movielens/datasets/genome_scores.csv"
genome_tags_fp = r"/Users/carolliu/Desktop/project_new/movielens/datasets/genome_tags.csv"
link_fp = r"/Users/carolliu/Desktop/project_new/movielens/datasets/link.csv"
movie_fp = r"/Users/carolliu/Desktop/project_new/movielens/datasets/movie.csv"
rating_fp = r"/Users/carolliu/Desktop/project_new/movielens/datasets/rating.csv"
tag_fp = r"/Users/carolliu/Desktop/project_new/movielens/datasets/tag.csv"

In [7]:
genome_scores = spark.read.csv(genome_scores_fp, header=True, inferSchema=True)
genome_tags = spark.read.csv(genome_tags_fp, header=True, inferSchema=True) 
link = spark.read.csv(link_fp, header=True, inferSchema=True)
movie = spark.read.csv(movie_fp, header=True, inferSchema=True)
rating = spark.read.csv(rating_fp, header=True, inferSchema=True)
tag = spark.read.csv(tag_fp, header=True, inferSchema=True)

In [8]:
# genome_scores.show()
# genome_scores.printSchema()

# genome_tags.show()
# genome_tags.printSchema()

# link.show()
# link.printSchema()

movie.show()
movie.printSchema()

rating.show()
rating.printSchema()

userList = rating.agg(collect_set('userId')).collect()
# print(userList)
# print(type(userList))

# tag.show()
# tag.printSchema()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [9]:
rating.agg({'timestamp': 'max'}).show()
rating.agg({'timestamp': 'min'}).show()
rating.agg({'rating': 'max'}).show()
rating.agg({'rating': 'min'}).show()
movie.count()

+-------------------+
|     max(timestamp)|
+-------------------+
|2015-03-31 06:40:02|
+-------------------+

+-------------------+
|     min(timestamp)|
+-------------------+
|1995-01-09 11:46:44|
+-------------------+

+-----------+
|max(rating)|
+-----------+
|        5.0|
+-----------+

+-----------+
|min(rating)|
+-----------+
|        0.5|
+-----------+



27278

In [10]:
movie_t = movie.withColumn("genre", explode(split("genres","\\|"))).withColumn("tag", lit(1))
movie_t.show()
movie_t.printSchema()
df2=movie_t.select(countDistinct("genre"))
df2.show()
genreList = movie_t.agg(collect_set('genre')).collect()
movieList = movie_t.agg(collect_set('movieId')).collect()

print(type(movieList))

+-------+--------------------+--------------------+---------+---+
|movieId|               title|              genres|    genre|tag|
+-------+--------------------+--------------------+---------+---+
|      1|    Toy Story (1995)|Adventure|Animati...|Adventure|  1|
|      1|    Toy Story (1995)|Adventure|Animati...|Animation|  1|
|      1|    Toy Story (1995)|Adventure|Animati...| Children|  1|
|      1|    Toy Story (1995)|Adventure|Animati...|   Comedy|  1|
|      1|    Toy Story (1995)|Adventure|Animati...|  Fantasy|  1|
|      2|      Jumanji (1995)|Adventure|Childre...|Adventure|  1|
|      2|      Jumanji (1995)|Adventure|Childre...| Children|  1|
|      2|      Jumanji (1995)|Adventure|Childre...|  Fantasy|  1|
|      3|Grumpier Old Men ...|      Comedy|Romance|   Comedy|  1|
|      3|Grumpier Old Men ...|      Comedy|Romance|  Romance|  1|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|   Comedy|  1|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|    Drama|  1|
|      4|W

# movie_t row to column

In [11]:
genre_list = ['Thriller', 'Action', 'Adventure', 'Crime', 'Comedy'
              , 'Horror', 'Film-Noir', 'Animation', '(no genres listed)', 'Mystery'
              , 'Documentary', 'Western', 'War', 'Fantasy', 'IMAX'
              , 'Drama', 'Children', 'Romance', 'Musical', 'Sci-Fi']
print(len(genre_list))

20


In [12]:
movie_genre = movie_t.withColumn("Thriller", expr("CASE WHEN genre = 'Thriller' THEN '1'" + 
                                                  "ELSE '0' END")).dropDuplicates(["movieId"])
movie_genre.show(truncate=False)

+-------+-------------------------------------+-------------------------------------------+---------+---+--------+
|movieId|title                                |genres                                     |genre    |tag|Thriller|
+-------+-------------------------------------+-------------------------------------------+---------+---+--------+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|Adventure|1  |0       |
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |Adventure|1  |0       |
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |Comedy   |1  |0       |
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |Comedy   |1  |0       |
|5      |Father of the Bride Part II (1995)   |Comedy                                     |Comedy   |1  |0       |
|6      |Heat (1995)                          |Action|Crime|Thriller            

In [13]:
movie_genre = movie_t.withColumn("Thriller", expr("CASE WHEN genre = 'Thriller' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Action", expr("CASE WHEN genre = 'Action' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Adventure", expr("CASE WHEN genre = 'Adventure' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Crime", expr("CASE WHEN genre = 'Crime' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Comedy", expr("CASE WHEN genre = 'Comedy' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Horror", expr("CASE WHEN genre = 'Horror' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Film-Noir", expr("CASE WHEN genre = 'Film-Noir' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Animation", expr("CASE WHEN genre = 'Animation' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("(no genres listed)", expr("CASE WHEN genre = '(no genres listed)' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Mystery", expr("CASE WHEN genre = 'Mystery' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Documentary", expr("CASE WHEN genre = 'Documentary' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Western", expr("CASE WHEN genre = 'Western' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("War", expr("CASE WHEN genre = 'War' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Fantasy", expr("CASE WHEN genre = 'Fantasy' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("IMAX", expr("CASE WHEN genre = 'IMAX' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Drama", expr("CASE WHEN genre = 'Drama' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Children", expr("CASE WHEN genre = 'Children' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Romance", expr("CASE WHEN genre = 'Romance' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Musical", expr("CASE WHEN genre = 'Musical' THEN '1'" + 
                                                  "ELSE '0' END")).withColumn("Sci-Fi", expr("CASE WHEN genre = 'Sci-Fi' THEN '1'" + 
                                                  "ELSE '0' END")).select(['movieId', 'title', 'Thriller', 'Action', 'Adventure', 'Crime', 'Comedy'
              , 'Horror', 'Film-Noir', 'Animation', '(no genres listed)', 'Mystery'
              , 'Documentary', 'Western', 'War', 'Fantasy', 'IMAX'
              , 'Drama', 'Children', 'Romance', 'Musical', 'Sci-Fi']).dropDuplicates(["movieId"])
                                                                             
movie_genre.show(truncate=False)

+-------+-------------------------------------+--------+------+---------+-----+------+------+---------+---------+------------------+-------+-----------+-------+---+-------+----+-----+--------+-------+-------+------+
|movieId|title                                |Thriller|Action|Adventure|Crime|Comedy|Horror|Film-Noir|Animation|(no genres listed)|Mystery|Documentary|Western|War|Fantasy|IMAX|Drama|Children|Romance|Musical|Sci-Fi|
+-------+-------------------------------------+--------+------+---------+-----+------+------+---------+---------+------------------+-------+-----------+-------+---+-------+----+-----+--------+-------+-------+------+
|1      |Toy Story (1995)                     |0       |0     |1        |0    |0     |0     |0        |0        |0                 |0      |0          |0      |0  |0      |0   |0    |0       |0      |0      |0     |
|2      |Jumanji (1995)                       |0       |0     |1        |0    |0     |0     |0        |0        |0                 |0   

# JOIN movie vs. rating

In [14]:
joined_t = rating.join(movie_t, rating.movieId == movie_t.movieId, 'leftouter')
joined_t.show()

+------+-------+------+-------------------+-------+--------------------+--------------------+---------+---+
|userId|movieId|rating|          timestamp|movieId|               title|              genres|    genre|tag|
+------+-------+------+-------------------+-------+--------------------+--------------------+---------+---+
|     1|      2|   3.5|2005-04-02 23:53:47|      2|      Jumanji (1995)|Adventure|Childre...|  Fantasy|  1|
|     1|      2|   3.5|2005-04-02 23:53:47|      2|      Jumanji (1995)|Adventure|Childre...| Children|  1|
|     1|      2|   3.5|2005-04-02 23:53:47|      2|      Jumanji (1995)|Adventure|Childre...|Adventure|  1|
|     1|     29|   3.5|2005-04-02 23:31:16|     29|City of Lost Chil...|Adventure|Drama|F...|   Sci-Fi|  1|
|     1|     29|   3.5|2005-04-02 23:31:16|     29|City of Lost Chil...|Adventure|Drama|F...|  Mystery|  1|
|     1|     29|   3.5|2005-04-02 23:31:16|     29|City of Lost Chil...|Adventure|Drama|F...|  Fantasy|  1|
|     1|     29|   3.5|2005-

In [15]:
user_genre = rating.join(movie_t, rating.movieId == movie_t.movieId, 'leftouter').withColumn('genre_ratings', (col("rating") * col("tag"))).groupBy(["userId", "genre"]).agg(f.sum("genre_ratings")).sort("userId", "genre")
user_genre.show()
user_genre.printSchema()

+------+---------+------------------+
|userId|    genre|sum(genre_ratings)|
+------+---------+------------------+
|     1|   Action|             246.0|
|     1|Adventure|             276.5|
|     1|Animation|              36.5|
|     1| Children|              68.5|
|     1|   Comedy|             153.0|
|     1|    Crime|              80.0|
|     1|    Drama|             162.0|
|     1|  Fantasy|             261.5|
|     1|   Horror|             168.5|
|     1|     IMAX|               8.5|
|     1|  Musical|              11.0|
|     1|  Mystery|              65.0|
|     1|  Romance|              43.5|
|     1|   Sci-Fi|             148.5|
|     1| Thriller|             158.0|
|     1|      War|              33.0|
|     1|  Western|              13.5|
|     2|   Action|              88.0|
|     2|Adventure|              82.0|
|     2|Animation|               3.0|
+------+---------+------------------+
only showing top 20 rows

root
 |-- userId: integer (nullable = true)
 |-- genre: string

In [16]:
windowval = (Window.partitionBy('userId'))
user_profile = user_genre.withColumn('userR_total', f.sum('sum(genre_ratings)').over(windowval)).withColumn('normed_ratings',(col("sum(genre_ratings)") / col("userR_total")))
user_profile.show()
user_profile.printSchema()

+------+---------+------------------+-----------+--------------------+
|userId|    genre|sum(genre_ratings)|userR_total|      normed_ratings|
+------+---------+------------------+-----------+--------------------+
|     1|   Action|             246.0|     1933.5| 0.12723041117145073|
|     1|Adventure|             276.5|     1933.5|  0.1430049133695371|
|     1|Animation|              36.5|     1933.5|0.018877682958365657|
|     1| Children|              68.5|     1933.5| 0.03542798034652185|
|     1|   Comedy|             153.0|     1933.5|  0.0791311093871218|
|     1|    Crime|              80.0|     1933.5| 0.04137574347039048|
|     1|    Drama|             162.0|     1933.5| 0.08378588052754073|
|     1|  Fantasy|             261.5|     1933.5| 0.13524696146883888|
|     1|   Horror|             168.5|     1933.5| 0.08714765968450995|
|     1|     IMAX|               8.5|     1933.5|0.004396172743728989|
|     1|  Musical|              11.0|     1933.5|0.005689164727178691|
|     

# User Nomarlized Ratings - convert row to column

In [17]:
user_profileM = user_profile.withColumn("Thriller", when(user_profile.genre == 'Thriller', user_profile.normed_ratings).otherwise(0))
user_profileM.show()

+------+---------+------------------+-----------+--------------------+------------------+
|userId|    genre|sum(genre_ratings)|userR_total|      normed_ratings|          Thriller|
+------+---------+------------------+-----------+--------------------+------------------+
|     1|   Action|             246.0|     1933.5| 0.12723041117145073|               0.0|
|     1|Adventure|             276.5|     1933.5|  0.1430049133695371|               0.0|
|     1|Animation|              36.5|     1933.5|0.018877682958365657|               0.0|
|     1| Children|              68.5|     1933.5| 0.03542798034652185|               0.0|
|     1|   Comedy|             153.0|     1933.5|  0.0791311093871218|               0.0|
|     1|    Crime|              80.0|     1933.5| 0.04137574347039048|               0.0|
|     1|    Drama|             162.0|     1933.5| 0.08378588052754073|               0.0|
|     1|  Fantasy|             261.5|     1933.5| 0.13524696146883888|               0.0|
|     1|  

In [18]:
user_profileM = user_profile.withColumn("Thriller", when(user_profile.genre == 'Thriller'
, user_profile.normed_ratings).otherwise(0)).withColumn("Action"
                                                        , when(user_profile.genre == 'Action', user_profile.normed_ratings).otherwise(0)).withColumn("Adventure"
                                                        , when(user_profile.genre == 'Adventure', user_profile.normed_ratings).otherwise(0)).withColumn("Crime"
                                                        , when(user_profile.genre == 'Crime', user_profile.normed_ratings).otherwise(0)).withColumn("Comedy"
                                                        , when(user_profile.genre == 'Comedy', user_profile.normed_ratings).otherwise(0)).withColumn("Action"
                                                        , when(user_profile.genre == 'Action', user_profile.normed_ratings).otherwise(0)).withColumn("Horror"
                                                        , when(user_profile.genre == 'Horror', user_profile.normed_ratings).otherwise(0)).withColumn("Film-Noir"
                                                        , when(user_profile.genre == 'Film-Noir', user_profile.normed_ratings).otherwise(0)).withColumn("Animation"
                                                        , when(user_profile.genre == 'Animation', user_profile.normed_ratings).otherwise(0)).withColumn("(no genres listed)"
                                                        , when(user_profile.genre == '(no genres listed)', user_profile.normed_ratings).otherwise(0)).withColumn("Mystery"
                                                        , when(user_profile.genre == 'Mystery', user_profile.normed_ratings).otherwise(0)).withColumn("Documentary"
                                                        , when(user_profile.genre == 'Documentary', user_profile.normed_ratings).otherwise(0)).withColumn("Western"
                                                        , when(user_profile.genre == 'Western', user_profile.normed_ratings).otherwise(0)).withColumn("War"
                                                        , when(user_profile.genre == 'War', user_profile.normed_ratings).otherwise(0)).withColumn("Fantasy"
                                                        , when(user_profile.genre == 'Fantasy', user_profile.normed_ratings).otherwise(0)).withColumn("IMAX"
                                                        , when(user_profile.genre == 'IMAX', user_profile.normed_ratings).otherwise(0)).withColumn("Drama"
                                                        , when(user_profile.genre == 'Drama', user_profile.normed_ratings).otherwise(0)).withColumn("Children"
                                                        , when(user_profile.genre == 'Children', user_profile.normed_ratings).otherwise(0)).withColumn("Romance"
                                                        , when(user_profile.genre == 'Romance', user_profile.normed_ratings).otherwise(0)).withColumn("Musical"
                                                        , when(user_profile.genre == 'Musical', user_profile.normed_ratings).otherwise(0)).withColumn("Sci-Fi"
                                                        , when(user_profile.genre == 'Sci-Fi', user_profile.normed_ratings).otherwise(0)).dropDuplicates(["userId"])
user_profileM.select(["userId","Thriller"]).show()

+------+--------+
|userId|Thriller|
+------+--------+
|     1|     0.0|
|     6|     0.0|
|    12|     0.0|
|    13|     0.0|
|    16|     0.0|
|    22|     0.0|
|    26|     0.0|
|    27|     0.0|
|    28|     0.0|
|    31|     0.0|
|    34|     0.0|
|    44|     0.0|
|    47|     0.0|
|    52|     0.0|
|    53|     0.0|
|    65|     0.0|
|    76|     0.0|
|    78|     0.0|
|    81|     0.0|
|    85|     0.0|
+------+--------+
only showing top 20 rows



In [19]:
cols = ("genre","sum(genre_ratings)","userR_total","normed_ratings")
user_profileM = user_profileM.drop(*cols)

In [20]:
user_profileM.show(truncate=False)

+------+--------+--------------------+---------+-----+------+------+---------+---------+------------------+-------+-----------+-------+---+-------+----+-----+--------+-------+-------+------+
|userId|Thriller|Action              |Adventure|Crime|Comedy|Horror|Film-Noir|Animation|(no genres listed)|Mystery|Documentary|Western|War|Fantasy|IMAX|Drama|Children|Romance|Musical|Sci-Fi|
+------+--------+--------------------+---------+-----+------+------+---------+---------+------------------+-------+-----------+-------+---+-------+----+-----+--------+-------+-------+------+
|1     |0.0     |0.12723041117145073 |0.0      |0.0  |0.0   |0.0   |0.0      |0.0      |0.0               |0.0    |0.0        |0.0    |0.0|0.0    |0.0 |0.0  |0.0     |0.0    |0.0    |0.0   |
|6     |0.0     |0.10683760683760683 |0.0      |0.0  |0.0   |0.0   |0.0      |0.0      |0.0               |0.0    |0.0        |0.0    |0.0|0.0    |0.0 |0.0  |0.0     |0.0    |0.0    |0.0   |
|12    |0.0     |0.14285714285714285 |0.0    

# take the userId=1 as an example 

In [21]:
#Method I Matrix
up_1 = user_profileM.filter(user_profileM.userId == "1")
up_1.show(truncate=False)
um_1 = rating.filter(rating.userId == "1").select(["userId","movieId"])
um_1.show(truncate=False)

+------+--------+-------------------+---------+-----+------+------+---------+---------+------------------+-------+-----------+-------+---+-------+----+-----+--------+-------+-------+------+
|userId|Thriller|Action             |Adventure|Crime|Comedy|Horror|Film-Noir|Animation|(no genres listed)|Mystery|Documentary|Western|War|Fantasy|IMAX|Drama|Children|Romance|Musical|Sci-Fi|
+------+--------+-------------------+---------+-----+------+------+---------+---------+------------------+-------+-----------+-------+---+-------+----+-----+--------+-------+-------+------+
|1     |0.0     |0.12723041117145073|0.0      |0.0  |0.0   |0.0   |0.0      |0.0      |0.0               |0.0    |0.0        |0.0    |0.0|0.0    |0.0 |0.0  |0.0     |0.0    |0.0    |0.0   |
+------+--------+-------------------+---------+-----+------+------+---------+---------+------------------+-------+-----------+-------+---+-------+----+-----+--------+-------+-------+------+

+------+-------+
|userId|movieId|
+------+-------

In [ ]:
# #Method II Pure Table
# up_114371 = user_profile.filter(user_profile.userId == "114371").select(["userId","genre","normed_ratings"])
# up_114371.show(truncate=False)
# um_114371 = rating.filter(rating.userId == "114371").select(["userId","movieId"])
# um_114371.show(truncate=False)

# Movies that this specific user did not watch

In [22]:
# Method I Matrix
umNM_1 = movie_genre.join(um_1, movie_genre.movieId == um_1.movieId, 'anti')
# umNM_1 = movie_genre.join(um_1, movie_genre.movieId == um_1.movieId, 'anti').drop("title")
umNM_1.show(truncate=False)

+-------+-------------------------------------+--------+------+---------+-----+------+------+---------+---------+------------------+-------+-----------+-------+---+-------+----+-----+--------+-------+-------+------+
|movieId|title                                |Thriller|Action|Adventure|Crime|Comedy|Horror|Film-Noir|Animation|(no genres listed)|Mystery|Documentary|Western|War|Fantasy|IMAX|Drama|Children|Romance|Musical|Sci-Fi|
+-------+-------------------------------------+--------+------+---------+-----+------+------+---------+---------+------------------+-------+-----------+-------+---+-------+----+-----+--------+-------+-------+------+
|1      |Toy Story (1995)                     |0       |0     |1        |0    |0     |0     |0        |0        |0                 |0      |0          |0      |0  |0      |0   |0    |0       |0      |0      |0     |
|3      |Grumpier Old Men (1995)              |0       |0     |0        |0    |1     |0     |0        |0        |0                 |0   

In [ ]:
# # Method II Simple table
# umNM_114371 = movie_genre.join(um_114371, movie_genre.movieId == um_114371.movieId, 'anti')
# upM_114371 = user_profileM.filter(user_profile.userId == "114371")
# upM_114371.select(["userId", "Thriller"]).show()

# Calculate Potential Ranking Score for Movies the user did not watch

In [23]:
# print((df.count(), len(df.columns)))
print(umNM_1.count(),len(umNM_1.columns))
print(up_1.count(),len(up_1.columns))
print(umNM_1.columns)
print(up_1.columns)

27103 22
1 21
['movieId', 'title', 'Thriller', 'Action', 'Adventure', 'Crime', 'Comedy', 'Horror', 'Film-Noir', 'Animation', '(no genres listed)', 'Mystery', 'Documentary', 'Western', 'War', 'Fantasy', 'IMAX', 'Drama', 'Children', 'Romance', 'Musical', 'Sci-Fi']
['userId', 'Thriller', 'Action', 'Adventure', 'Crime', 'Comedy', 'Horror', 'Film-Noir', 'Animation', '(no genres listed)', 'Mystery', 'Documentary', 'Western', 'War', 'Fantasy', 'IMAX', 'Drama', 'Children', 'Romance', 'Musical', 'Sci-Fi']


In [24]:
# user profile
uv_1 = np.array(up_1.select('Thriller', 'Action', 'Adventure', 'Crime', 'Comedy'
                          , 'Horror', 'Film-Noir', 'Animation', '(no genres listed)', 'Mystery'
                          , 'Documentary', 'Western', 'War', 'Fantasy', 'IMAX'
                          , 'Drama', 'Children', 'Romance', 'Musical', 'Sci-Fi').collect()).T
uv_1 = np.asarray(uv_1, dtype='float64')
# candidate movies
cm_1 = np.array(umNM_1.select('Thriller', 'Action', 'Adventure', 'Crime', 'Comedy'
                          , 'Horror', 'Film-Noir', 'Animation', '(no genres listed)', 'Mystery'
                          , 'Documentary', 'Western', 'War', 'Fantasy', 'IMAX'
                          , 'Drama', 'Children', 'Romance', 'Musical', 'Sci-Fi').collect())

cm_1 = np.asarray(cm_1, dtype='float64')
print(uv_1.shape)
print(cm_1.shape)
print(uv_1)
print(cm_1)

(20, 1)
(27103, 20)
[[0.        ]
 [0.12723041]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [25]:
cmr_1 = np.matmul(cm_1,uv_1)
cmr_1.shape

(27103, 1)

In [26]:
cmr_1

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [27]:
df_cmr_1 = pd.DataFrame(cmr_1)
spark.conf.set("spark.sql.execution.arrow.enabled","true")
dfsk_cmr_1 = spark.createDataFrame(df_cmr_1).selectExpr("0 as ratings").withColumn("row_idx",row_number().over(Window.orderBy(monotonically_increasing_id())))
dfsk_cmr_1.show()
print(dfsk_cmr_1.count(),len(dfsk_cmr_1.columns))

/usr/local/opt/apache-spark/libexec/python/pyspark/sql/pandas/conversion.py:329: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 1.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


+-------+-------+
|ratings|row_idx|
+-------+-------+
|      0|      1|
|      0|      2|
|      0|      3|
|      0|      4|
|      0|      5|
|      0|      6|
|      0|      7|
|      0|      8|
|      0|      9|
|      0|     10|
|      0|     11|
|      0|     12|
|      0|     13|
|      0|     14|
|      0|     15|
|      0|     16|
|      0|     17|
|      0|     18|
|      0|     19|
|      0|     20|
+-------+-------+
only showing top 20 rows

27103 2


In [28]:
movieNList_1 = umNM_1.select("movieId")
# print(type(movieNList_1))
movieNList_1.show()
print(movieNList_1.count(),len(movieNList_1.columns))

+-------+
|movieId|
+-------+
|    148|
|    463|
|    471|
|    496|
|    833|
|   1088|
|   1238|
|   1342|
|   1580|
|   1591|
|   1645|
|   1829|
|   1959|
|   2122|
|   2142|
|   2366|
|   2659|
|   2866|
|   3175|
|   3749|
+-------+
only showing top 20 rows

27103 1


In [29]:
movieNList_1 = movieNList_1.withColumn("row_idx",row_number().over(Window.orderBy(monotonically_increasing_id())))
movieNList_1.show()

+-------+-------+
|movieId|row_idx|
+-------+-------+
|    148|      1|
|    463|      2|
|    471|      3|
|    496|      4|
|    833|      5|
|   1088|      6|
|   1238|      7|
|   1342|      8|
|   1580|      9|
|   1591|     10|
|   1645|     11|
|   1829|     12|
|   1959|     13|
|   2122|     14|
|   2142|     15|
|   2366|     16|
|   2659|     17|
|   2866|     18|
|   3175|     19|
|   3749|     20|
+-------+-------+
only showing top 20 rows



In [31]:
movieRC_1 = movieNList_1.join(dfsk_cmr_1, dfsk_cmr_1.row_idx==movieNList_1.row_idx, 'left')
movieRC_1.show()
print(movieRC_1.count(),len(movieRC_1.columns))

+-------+-------+-------+-------+
|movieId|row_idx|ratings|row_idx|
+-------+-------+-------+-------+
|    148|      1|      0|      1|
|    463|      2|      0|      2|
|    471|      3|      0|      3|
|    496|      4|      0|      4|
|    833|      5|      0|      5|
|   1088|      6|      0|      6|
|   1238|      7|      0|      7|
|   1342|      8|      0|      8|
|   1580|      9|      0|      9|
|   1591|     10|      0|     10|
|   1645|     11|      0|     11|
|   1829|     12|      0|     12|
|   1959|     13|      0|     13|
|   2122|     14|      0|     14|
|   2142|     15|      0|     15|
|   2366|     16|      0|     16|
|   2659|     17|      0|     17|
|   2866|     18|      0|     18|
|   3175|     19|      0|     19|
|   3749|     20|      0|     20|
+-------+-------+-------+-------+
only showing top 20 rows

27103 4


In [32]:
movieRC_1.sort(col("ratings").desc()).show(truncate=False)

+-------+-------+-------+-------+
|movieId|row_idx|ratings|row_idx|
+-------+-------+-------+-------+
|148    |1      |0      |1      |
|463    |2      |0      |2      |
|471    |3      |0      |3      |
|496    |4      |0      |4      |
|833    |5      |0      |5      |
|1088   |6      |0      |6      |
|1238   |7      |0      |7      |
|1342   |8      |0      |8      |
|1580   |9      |0      |9      |
|1591   |10     |0      |10     |
|1645   |11     |0      |11     |
|1829   |12     |0      |12     |
|1959   |13     |0      |13     |
|2122   |14     |0      |14     |
|2142   |15     |0      |15     |
|2366   |16     |0      |16     |
|2659   |17     |0      |17     |
|2866   |18     |0      |18     |
|3175   |19     |0      |19     |
|3749   |20     |0      |20     |
+-------+-------+-------+-------+
only showing top 20 rows



In [ ]:
class userRec:
    def __init__(self):
        self.movie = np.array()
        